In [197]:
import pandas as pd
import requests, json
from time import sleep

## Lista Produtos

In [201]:
produtos = list()
data = {'formato': 'json',
        'token': '2b6fc7102240cedcc9166c43921ea73eea82b876',
        'pesquisa': '',
        'pagina': 1}
while True:
    session = requests.post(url='https://api.tiny.com.br/api2/produtos.pesquisa.php', data=data).json()
    data['pagina'] += 1
    try:
        produtos += [x['produto'] for x in session['retorno']['produtos']]
    except:
        break
produtos = pd.DataFrame(produtos)

Apurar databaset completo de produtos

In [199]:
produtos_completos = list()
for id in produtos['id']:
    data = {
        'formato': 'json',
        'token': '2b6fc7102240cedcc9166c43921ea73eea82b876',
        'id': id
    }
    try:
        produtos_completos += [requests.post(url='https://api.tiny.com.br/api2/produto.obter.php', data=data).json()['retorno']['produto']]
    except:
        sleep(60)
        produtos_completos += [requests.post(url='https://api.tiny.com.br/api2/produto.obter.php', data=data).json()['retorno']['produto']]
produtos_completos = pd.DataFrame(produtos_completos)

Filtragem de dados

In [204]:
produtos_filtrados = produtos_completos[pd.DataFrame([produtos_completos['categoria'] == x for x in ['Bovinos', 'Suínos', 'Aves', 'Especiais']]).any()]
produtos_filtrados = produtos_filtrados[produtos_filtrados['slug'] != '']

Limpeza de dados

In [209]:
produtos_formatados = produtos_filtrados.drop(['preco', 'marca', 'preco_promocional', 'ncm', 'origem', 'gtin', 'gtin_embalagem', 'localizacao', 'peso_liquido', 'peso_bruto', 'estoque_maximo', 'id_fornecedor', 'codigo_fornecedor', 'codigo_pelo_fornecedor', 'unidade_por_caixa', 'preco_custo', 'preco_custo_medio', 'situacao', 'tipo', 'classe_ipi', 'valor_ipi_fixo', 'cod_lista_servicos', 'descricao_complementar', 'garantia', 'cest', 'obs', 'tipoVariacao', 'variacoes', 'idProdutoPai', 'sob_encomenda', 'dias_preparacao', 'tipoEmbalagem', 'alturaEmbalagem', 'comprimentoEmbalagem', 'larguraEmbalagem', 'diametroEmbalagem', 'categoria', 'anexos', 'imagens_externas', 'classe_produto', 'seo_title', 'seo_keywords', 'link_video', 'seo_description', 'kit'], axis=1)

# unidade
produtos_formatados['unidade'] = produtos_formatados['unidade'].str.casefold()
produtos_formatados['unidade'] = ['un' if x != 'kg' else 'kg' for x in produtos_formatados['unidade']]

produtos_formatados

,id,nome,codigo,unidade,estoque_minimo,slug
1,623449734,ALCATRA COM QUEIJO,ALCAQP,kg,8.62,Alcatra
2,623449795,ALMÔNDEGA BACON,ALMOBARB,kg,5.00,Almôndega
3,623449782,ALMÔNDEGA QUEIJO,ALMOQRB,kg,4.50,Almôndega
4,778208483,ANCHO ALIMENTAR,ANCHALI,kg,0.00,Ancho
5,646749750,ANCHO DI PRIMA,ANCHDIPDI,kg,0.00,Ancho
...,...,...,...,...,...,...
143,638867885,T-BONE MINERVA ANGUS,TBONMINE,kg,5.00,T-Bone
144,640758272,T-BONE RARO,TBNERARO,kg,0.00,T-Bone
146,623450791,TOMAHAWK CARA PRETA,TOMACP,kg,2.00,Tomahawk
147,647280670,TOMAHAWK RARO,TOMARARO,kg,0.00,Tomahawk


Conversão para POST

In [224]:
produtos_post = produtos_formatados.to_dict(orient='records')
json.dumps(produtos_post)

'[{"id": "623449734", "nome": "ALCATRA COM QUEIJO", "codigo": "ALCAQP", "unidade": "kg", "estoque_minimo": 8.62, "slug": "Alcatra", "estoque": 14.49}, {"id": "623449795", "nome": "ALM\\u00d4NDEGA BACON", "codigo": "ALMOBARB", "unidade": "kg", "estoque_minimo": 5.0, "slug": "Alm\\u00f4ndega", "estoque": 0}, {"id": "623449782", "nome": "ALM\\u00d4NDEGA QUEIJO", "codigo": "ALMOQRB", "unidade": "kg", "estoque_minimo": 4.5, "slug": "Alm\\u00f4ndega", "estoque": 0}, {"id": "778208483", "nome": "ANCHO ALIMENTAR", "codigo": "ANCHALI", "unidade": "kg", "estoque_minimo": 0.0, "slug": "Ancho", "estoque": 0}, {"id": "646749750", "nome": "ANCHO DI PRIMA", "codigo": "ANCHDIPDI", "unidade": "kg", "estoque_minimo": 0.0, "slug": "Ancho", "estoque": 0}, {"id": "729652533", "nome": "ANCHO MINERVA", "codigo": "ANCHMINERV", "unidade": "kg", "estoque_minimo": 0.0, "slug": "Ancho", "estoque": 0}, {"id": "638403161", "nome": "ANCHO MONTANA STEAKHOUSE", "codigo": "ANCHMOSTKMA", "unidade": "kg", "estoque_minimo

## Estoque

In [ ]:
produtos_estoque = produtos_formatados
estoque = list()
for id in produtos_estoque['id']:
    data = {
        'formato': 'json',
        'token': '2b6fc7102240cedcc9166c43921ea73eea82b876',
        'id': id
    }
    try:
        estoque += [requests.post(url='https://api.tiny.com.br/api2/produto.obter.estoque.php', data=data).json()['retorno']['produto']['saldo']]
    except:
        sleep(60)
        estoque += [requests.post(url='https://api.tiny.com.br/api2/produto.obter.estoque.php', data=data).json()['retorno']['produto']]
produtos_estoque['estoque'] = estoque
produtos_estoque.drop(['codigo', 'unidade', 'estoque_minimo', 'slug'], axis=1, inplace=True)

Conversão para POST

In [231]:
estoque_post = produtos_estoque.to_dict(orient='records')
json.dumps(estoque_post)

'[{"id": "623449734", "nome": "ALCATRA COM QUEIJO", "estoque": 14.49}, {"id": "623449795", "nome": "ALM\\u00d4NDEGA BACON", "estoque": 0}, {"id": "623449782", "nome": "ALM\\u00d4NDEGA QUEIJO", "estoque": 0}, {"id": "778208483", "nome": "ANCHO ALIMENTAR", "estoque": 0}, {"id": "646749750", "nome": "ANCHO DI PRIMA", "estoque": 0}, {"id": "729652533", "nome": "ANCHO MINERVA", "estoque": 0}, {"id": "638403161", "nome": "ANCHO MONTANA STEAKHOUSE", "estoque": 20.01}, {"id": "623449824", "nome": "ASINHA RECHEADA", "estoque": 7}, {"id": "623449837", "nome": "ASSADO DE TIRAS ARG", "estoque": 0}, {"id": "661843960", "nome": "BACON FATIADO", "estoque": 0}, {"id": "649329262", "nome": "BUTCHER STEAK PORCAO", "estoque": 0}, {"id": "733149637", "nome": "CARPACCIO CARA PRETA", "estoque": 0}, {"id": "659216777", "nome": "CARR\\u00c9 CORDEIRO 1953", "estoque": 0}, {"id": "623449886", "nome": "CHORIZO EST\\u00c2NCIA 92", "estoque": 10.88}, {"id": "623449902", "nome": "CORA\\u00c7\\u00c3O", "estoque": 6.